In [2]:
import os

In [3]:
%pwd

'/home/pro/Documents/ML-DL-Project/Deep-Learning-Chicken-Disease-With-LLM/research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/home/pro/Documents/ML-DL-Project/Deep-Learning-Chicken-Disease-With-LLM'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    blob_name: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            blob_name=config.blob_name,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [10]:
# import boto3
import os
from dotenv import load_dotenv
# from botocore.exceptions import ClientError
from azure.storage.blob import BlobServiceClient
from azure.core.exceptions import ResourceNotFoundError
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size
import tempfile

load_dotenv()

True

In [14]:

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config



    def download_from_blob(self, container_name, blob_name, download_path, connection_string):
        try:
            print(f"Downloading '{blob_name}' from container '{container_name}' to '{download_path}'...")

            # Initialize the BlobServiceClient
            blob_service_client = BlobServiceClient.from_connection_string(connection_string)
            blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

            # Download the blob
            with open(download_path, "wb") as f:
                download_stream = blob_client.download_blob()
                f.write(download_stream.readall())

            print("Download complete.")

        except ResourceNotFoundError:
            print(f"Error: Blob '{blob_name}' not found in container '{container_name}'.")
        except Exception as e:
            print(f"Unexpected error: {e}")
            

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory specified by self.config.unzip_dir.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)

    container_name = os.getenv("STORAGE_ACCOUNT_CONTAINER")
    blob_name = "data.zip"
    object_key = "data.zip"

    download_path = data_ingestion_config.local_data_file  # already set in config
    connection_string = os.getenv("STORAGE_ACCOUNT_CONNECTION")

    data_ingestion.download_from_blob(container_name, blob_name, download_path, connection_string)

    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-10-26 15:30:53,539: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-10-26 15:30:53,546: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-26 15:30:53,551: INFO: common: created directory at: artifacts]
[2025-10-26 15:30:53,555: INFO: common: created directory at: artifacts/data_ingestion]
[2025-10-26 15:30:53,566: INFO: _universal: Request URL: 'http://127.0.0.1:10000/devstoreaccount1/chicken-disease/data.zip'
Request method: 'GET'
Request headers:
    'x-ms-range': 'REDACTED'
    'x-ms-version': 'REDACTED'
    'Accept': 'application/xml'
    'User-Agent': 'azsdk-python-storage-blob/12.26.0 Python/3.8.20 (Linux-6.14.0-33-generic-x86_64-with-glibc2.17)'
    'x-ms-date': 'REDACTED'
    'x-ms-client-request-id': '77cdd452-b24e-11f0-8938-2016b929d5dd'
    'Authorization': 'REDACTED'
No body was attached to the request]
[2025-10-26 15:30:53,608: INFO: _universal: Response status: 206
Response headers:
    'Server': 'Azurite-Blob/3.35.0'
    'las

Download complete.
